In [4]:
import pandas as pd
import tushare as ts
import os
import time, datetime
import threadpool
import StkDataKeeper as sk

def getDailyData():
    # create folder
    cwd = os.getcwd()
    today_date = time.strftime("%Y%m%d")
    target_dir = today_date
    # target_dir = os.path.join("/home/samx/ramdisk/", today_date)
    try:
        os.makedirs(name=target_dir)
    except:
        print ('folder already existed')

    # clean up the index with numbers, and make date the default index
    def pre_process(df):
        df.set_index('date', inplace=True)
        # df.drop(df.columns[0], axis=1, inplace=True)
        return df

    def obtain_daily_data(sk_code, target_dir = ""):
        print ("Processing: "+sk_code)

        if target_dir!="":
            target_dir=target_dir+os.path.sep

        day_df = pre_process(ts.get_k_data(sk_code))
        csv_filename = target_dir + "day_" + sk_code + ".csv"
        day_df.to_csv(csv_filename)
        
        print (day_df.head(1))

        five_min_df = pre_process(ts.get_k_data(sk_code, ktype='5'))
        csv_filename = target_dir + "5min_" + sk_code + ".csv"
        five_min_df.to_csv(csv_filename)

        print (five_min_df.head(1))

    # obtain stock list
    try:
        stock_basics = ts.get_stock_basics()
        code_series = stock_basics.index
    except:
        keeper = sk.StkDataKeeper()
        keeper.load_stock_list("stock_basics.csv")
        code_series = keeper.stock_list

    # prepare arguments
    args_list=[]
    for code in code_series:
        args_list.append(([code, target_dir], None))

    # collect
    pool = threadpool.ThreadPool(30)
    requests = threadpool.makeRequests(obtain_daily_data, args_list)
    [pool.putRequest(req) for req in requests]
    pool.wait()

    print ('done')


def get_daily_chart(analysis_type = -1):
    # determine 涨停
    def isZhangting(s_df, index=-1):
        if (s_df.iloc[index].close/s_df.iloc[index-1].close) >= 1.099:
            return True
        else:
            return False

    #determine 上涨
    def isShangzhang(s_df, index=-1):
        if len(s_df.index) >= (index)+1:
            return ((s_df.iloc[index].close/s_df.iloc[index-1].close-1) > 0.02)
        else:
            return False

    #determine 条件一
    def isQualifiedCondition1(s_df, index=-1):
        if len(stock_df.index)>=(abs(index)+2):
            if (s_df.iloc[index].close * s_df.iloc[index].volume > 200000):
                return  (isZhangting(s_df, index-1) and isShangzhang(s_df,index))
        return False

    #determine 条件二
    def isQualifiedCondition2(s_df, index=-1):
        if len(stock_df.index)>=(abs(index)+3):
            if (s_df.iloc[index].close / s_df.iloc[index-2].close < 0.92) \
                    and (s_df.iloc[index].close<s_df.iloc[index-1].close) \
                    and (s_df.iloc[index-1].close<s_df.iloc[index-2].close)\
                    and isZhangting(s_df, index-2):
                return True
        return False

    #determine 条件三
    def isQualifiedCondition3(s_df, index=-1):
        if len(stock_df.index)>=(abs(index)+3): 
            if isQualifiedCondition1(s_df, index-1): 
                if (s_df.iloc[index].close / s_df.iloc[index-1].close < 0.95):
                    return True
            return False

    def completeCode(code_str):
        if len(code_str)<6:
            return "0"*(6-len(code_str))+code_str
        else:
            return code_str

    print (time.strftime("%M:%S"))

    ########################################
    datafolder_dir = os.getcwd()+"/current"
    ########################################

    file_list = os.listdir(datafolder_dir)

    trading = -2
    post_analysis = -1

    analysis_type = post_analysis

    file_index = 0
    select_list = []
    file_dict = {}
    for file in file_list:
        #print ("Working on file %s: %s"%(file_index, file))
        file_index = file_index + 1
        if ("day" in file):
            stock_df = pd.read_csv(datafolder_dir+"/"+file)
            if isQualifiedCondition1(stock_df, analysis_type):
                select_list.append("condition 1:" + completeCode(str(stock_df.iloc[0].code)))
                file_dict[completeCode(str(stock_df.iloc[0].code))] = datafolder_dir+"/"+file
            if isQualifiedCondition2(stock_df, analysis_type):
                select_list.append("condition 2:" + completeCode(str(stock_df.iloc[0].code)))
                file_dict[completeCode(str(stock_df.iloc[0].code))] = datafolder_dir+"/"+file
            if isQualifiedCondition3(stock_df, analysis_type):
                select_list.append("condition 3:" + completeCode(str(stock_df.iloc[0].code)))
                file_dict[completeCode(str(stock_df.iloc[0].code))] = datafolder_dir+"/"+file

    select_list.sort()

    # ------------------------------
    for file in select_list:
        print (file)

    for index in range(len(select_list)):
        select_list[index] = select_list[index][-6:]

    print (select_list)

    stock_df_list = []
    for item in select_list:
        stock_df_list.append(pd.read_csv(file_dict[item]))

    # ------------------------------

    painter = sk.StkDiagram()
    keeper = sk.StkDataKeeper()
    try:
        keeper.get_stock_basics_tu("stock_basics.csv")
    except:
        keeper.load_stock_list("stock_basics.csv")
        
    keeper.load_stock_list("stock_basics.csv")
    klines = painter.paint_klines(select_list, "stock_basics.csv")

    klines.width = 1600
    klines.render("lyd_result/render.html")
    klines.width = 990
    klines
    
    # get data from yesterday
    analysis_type = trading

    file_index = 0
    select_list = []
    file_dict = {}
    for file in file_list:
        #print ("Working on file %s: %s"%(file_index, file))
        file_index = file_index + 1
        if ("day" in file):
            stock_df = pd.read_csv(datafolder_dir+"/"+file)
            if isQualifiedCondition1(stock_df, analysis_type):
                select_list.append("condition 1:" + completeCode(str(stock_df.iloc[0].code)))
                file_dict[completeCode(str(stock_df.iloc[0].code))] = datafolder_dir+"/"+file
            if isQualifiedCondition2(stock_df, analysis_type):
                select_list.append("condition 2:" + completeCode(str(stock_df.iloc[0].code)))
                file_dict[completeCode(str(stock_df.iloc[0].code))] = datafolder_dir+"/"+file
            if isQualifiedCondition3(stock_df, analysis_type):
                select_list.append("condition 3:" + completeCode(str(stock_df.iloc[0].code)))
                file_dict[completeCode(str(stock_df.iloc[0].code))] = datafolder_dir+"/"+file

    select_list.sort()

    # ------------------------------
    for file in select_list:
        print (file)

    for index in range(len(select_list)):
        select_list[index] = select_list[index][-6:]

    print (select_list)

    stock_df_list = []
    for item in select_list:
        stock_df_list.append(pd.read_csv(file_dict[item]))

    # ------------------------------

    painter1 = sk.StkDiagram()
    keeper1 = sk.StkDataKeeper()
    keeper1.load_stock_list("stock_basics.csv")
    klines1 = painter1.paint_klines(select_list, "stock_basics.csv")

    klines1.width = 1600
    klines1.render("lyd_result/render1.html")
    klines1.width = 990
    klines1

In [2]:
while (True):
    now = datetime.datetime.now()
        
    while (now.weekday() == 5 or now.weekday() == 6):
        time.sleep(3600)
        now = datetime.datetime.now()
        
    today_end2 = datetime.datetime(now.year, now.month, now.day, 15, 1)
    tomorrow_end2 = today_end2 + datetime.timedelta(days=1)
    
    while (now-today_end2).days < 0: 
        time.sleep(3600)
    
    mark_begin = datetime.datetime.now()
    # execution--------------------------
    getDailyData() 
    # execution--------------------------
    mark_done_get = datetime.datetime.now()
    print("Obtain data in: "+str(mark_done_get- mark_begin))
    
    hist_dir = os.path.abspath("stk_new_update")
    daily_dir = os.path.abspath(time.strftime("%Y%m%d"))
    target_dir = os.path.abspath("stk_new_update")

    stk_keeper=sk.StkDataKeeper()
    stk_keeper.load_stock_list()
    stk_keeper.merge_hist_with_daily_folder(hist_dir, daily_dir, target_dir)
    print("Refresh data in: "+str(datetime.datetime.now() - mark_done_get))
    
    # paint two charts
    get_daily_chart()
    
    while (now-tomorrow_end2).days < 0: 
        time.sleep(3600)
        now = datetime.datetime.now() 

folder already existed
Processing: 603015Processing: 300746Processing: 600569Processing: 002161Processing: 002607Processing: 300549Processing: 002628






Processing: 300648Processing: 002782Processing: 300647Processing: 300682Processing: 603486Processing: 603045Processing: 002909Processing: 000532Processing: 603218Processing: 601619Processing: 002908Processing: 300745Processing: 000735Processing: 002733Processing: 600673Processing: 002619Processing: 600433Processing: 000860Processing: 603901Processing: 600622Processing: 300241Processing: 000613Processing: 300619






















            open  close   high   low  volume    code
date                                                
2017-10-20  9.92  10.92  10.92  9.92   108.0  002908
              open   close   high     low   volume    code
date                                                      
2015-10-13  12.008  12.147  12.27  11.874  85285.0  603901
             open  close   high    low    volume    code
date          

                  open  close   high    low  volume    code
date                                                       
2018-05-24 14:55  11.9   11.9  11.91  11.89  2201.0  000532
Processing: 300144
              open   close    high     low   volume    code
date                                                       
2015-04-07  13.608  14.311  14.624  13.576  81035.0  300267
                  open  close  high   low  volume    code
date                                                     
2018-05-24 14:55  5.38   5.41  5.41  5.36  4070.0  002628
Processing: 603876
              open   close   high    low   volume    code
date                                                     
2015-10-26  31.483  31.424  31.73  30.25  95088.0  002111
             open  close   high    low  volume    code
date                                                  
2015-12-23  2.748  3.297  3.297  2.748   216.0  002785
                   open  close   high    low  volume    code
date                        

Processing: 601155
             open  close   high    low    volume    code
date                                                    
2015-10-14  2.606  2.582  2.641  2.577  221673.0  600782
                   open  close  high    low  volume    code
date                                                       
2018-05-24 14:55  27.36  27.38  27.4  27.33   380.0  002800
Processing: 601360
             open  close   high   low   volume    code
date                                                  
2014-04-15  1.775  1.765  1.835  1.75  28389.0  600725
              open   close    high     low    volume    code
date                                                        
2015-02-04  11.751  12.153  12.555  11.671  141463.0  300056
            open  close  high   low    volume    code
date                                                 
2014-11-04  7.59   7.63  7.72  7.55  201310.0  600877
                   open  close   high   low  volume    code
date                                     

                  open  close  high   low  volume    code
date                                                     
2018-02-28 14:55  7.02   7.02  7.02  7.02   490.0  300056
Processing: 002917
                  open  close  high  low  volume    code
date                                                    
2018-05-24 14:55  5.55    5.5  5.55  5.5  3726.0  600877
Processing: 000710
             open  close   high    low   volume    code
date                                                   
2015-02-16  3.402  3.421  3.439  3.365  50218.0  601003
             open   close    high     low    volume    code
date                                                       
2015-10-26  27.75  29.831  29.831  26.012  278311.0  002364
             open  close   high    low  volume    code
date                                                  
2017-08-04  11.27  13.52  13.52  11.27   122.0  300686
                  open  close  high  low  volume    code
date                                           

                   open  close   high    low  volume    code
date                                                        
2018-05-24 14:55  36.14  35.98  36.15  35.98   852.0  300657
Processing: 300099
             open  close   high   low   volume    code
date                                                  
2015-03-19  17.19   16.7  17.19  16.5  44111.0  000710
             open  close   high    low    volume    code
date                                                    
2015-10-15  5.663   5.87  5.992  5.516  105693.0  002157
                   open  close   high    low  volume    code
date                                                        
2018-05-24 14:55  11.42  11.42  11.42  11.42   292.0  603069
Processing: 300390
                   open  close   high    low  volume    code
date                                                        
2018-05-24 14:55  63.79   63.3  63.79  63.09  4245.0  002931
Processing: 600462
            open  close   high    low   volume    code
dat

...

             open   close   high     low    volume    code
date                                                      
2015-04-30  14.71  14.469  15.12  14.464  431781.0  002465Processing: 601138
Processing: 601066

                   open  close   high    low  volume    code
date                                                        
2018-05-22 14:55  33.64  33.64  33.64  33.64   279.0  002828
Processing: 600680
Processing: 600401
                   open  close   high    low   volume    code
date                                                         
2018-05-24 14:55  13.12   13.1  13.12  13.04  13134.0  002229
Processing: 300750
Processing: 300747
             open  close  high    low  volume    code
date                                                 
2017-01-25  6.382   7.66  7.66  6.382    57.0  300578
            open   close    high   low  volume    code
date                                                  
2015-06-11  9.09  10.906  10.906  9.09    55.0  300472
             

  File "/home/samx/miniconda/lib/python3.6/site-packages/pandas/core/frame.py", line 3146, in set_index
    level = frame[col]._values
  File "/home/samx/miniconda/lib/python3.6/site-packages/pandas/core/frame.py", line 2139, in __getitem__
    return self._getitem_column(key)
  File "/home/samx/miniconda/lib/python3.6/site-packages/pandas/core/frame.py", line 2146, in _getitem_column
    return self._get_item_cache(key)
  File "/home/samx/miniconda/lib/python3.6/site-packages/pandas/core/generic.py", line 1842, in _get_item_cache
    values = self._data.get(item)
  File "/home/samx/miniconda/lib/python3.6/site-packages/pandas/core/internals.py", line 3838, in get
    loc = self.items.get_loc(item)
  File "/home/samx/miniconda/lib/python3.6/site-packages/pandas/core/indexes/base.py", line 2524, in get_loc
    return self._engine.get_loc(self._maybe_cast_indexer(key))
  File "pandas/_libs/index.pyx", line 117, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/index.pyx", li

done
Obtain data in: 0:03:12.155755
['', '/home/samx/notebooks/20180607/day_601360.csv', '/home/samx/notebooks/stk_new_update/SH601360.csv']
['', '/home/samx/notebooks/20180607/day_300504.csv', '/home/samx/notebooks/stk_new_update/SZ300504.csv']
['', '/home/samx/notebooks/20180607/day_600929.csv', '/home/samx/notebooks/stk_new_update/SH600929.csv']
['', '/home/samx/notebooks/20180607/day_300746.csv', '/home/samx/notebooks/stk_new_update/SZ300746.csv']
['', '/home/samx/notebooks/20180607/day_300737.csv', '/home/samx/notebooks/stk_new_update/SZ300737.csv']
['', '/home/samx/notebooks/20180607/day_603045.csv', '/home/samx/notebooks/stk_new_update/SH603045.csv']
['', '/home/samx/notebooks/20180607/day_603080.csv', '/home/samx/notebooks/stk_new_update/SH603080.csv']
['', '/home/samx/notebooks/20180607/day_300634.csv', '/home/samx/notebooks/stk_new_update/SZ300634.csv']
['', '/home/samx/notebooks/20180607/day_002921.csv', '/home/samx/notebooks/stk_new_update/SZ002921.csv']
['', '/home/samx/no

['', '/home/samx/notebooks/20180607/day_000828.csv', '/home/samx/notebooks/stk_new_update/SZ000828.csv']
['', '/home/samx/notebooks/20180607/day_002117.csv', '/home/samx/notebooks/stk_new_update/SZ002117.csv']
['', '/home/samx/notebooks/20180607/day_002732.csv', '/home/samx/notebooks/stk_new_update/SZ002732.csv']
['', '/home/samx/notebooks/20180607/day_000557.csv', '/home/samx/notebooks/stk_new_update/SZ000557.csv']
['', '/home/samx/notebooks/20180607/day_000998.csv', '/home/samx/notebooks/stk_new_update/SZ000998.csv']
['', '/home/samx/notebooks/20180607/day_000518.csv', '/home/samx/notebooks/stk_new_update/SZ000518.csv']
['', '/home/samx/notebooks/20180607/day_603517.csv', '/home/samx/notebooks/stk_new_update/SH603517.csv']
['', '/home/samx/notebooks/20180607/day_002872.csv', '/home/samx/notebooks/stk_new_update/SZ002872.csv']
['', '/home/samx/notebooks/20180607/day_600463.csv', '/home/samx/notebooks/stk_new_update/SH600463.csv']
['', '/home/samx/notebooks/20180607/day_300169.csv', '/

EmptyDataError: No columns to parse from file

In [2]:
getDailyData()
get_daily_chart()

Processing: 000571
Processing: 002799Processing: 000921Processing: 300455Processing: 002917



Processing: 002290
Processing: 300109
Processing: 002693
Processing: 300745Processing: 603045Processing: 300501Processing: 600579



Processing: 300710Processing: 300486Processing: 603486Processing: 002453Processing: 002892
Processing: 300647



Processing: 603577
Processing: 603386Processing: 600309Processing: 600983Processing: 000062Processing: 002607Processing: 300746Processing: 600960Processing: 600230Processing: 300250Processing: 002228Processing: 002889











             open  close   high    low  volume    code
date                                                  
2017-08-17  21.07  25.29  25.29  21.07    86.0  002892
             open  close   high    low  volume    code
date                                                  
2018-05-23  15.38  16.78  16.78  15.38   107.0  300745
             open  close   high    low    volume    code
date                                       

                   open  close   high    low  volume    code
date                                                        
2018-05-21 14:55  16.82  16.82  16.82  16.82     0.0  603045                  open  close   high    low  volume    code
date                                                       
2018-05-21 14:55  42.0  41.97  42.01  41.92  3743.0  600230

Processing: 002672Processing: 603157

                   open  close  high    low  volume    code
date                                                       
2018-05-21 14:55  13.58  13.57  13.6  13.57  1166.0  603577
Processing: 002282
                   open  close   high    low  volume    code
date                                                        
2018-05-21 14:55  30.47  30.54  30.56  30.46  1479.0  002917
              open   close    high     low  volume    code
date                                                      
2016-11-01  12.109  13.211  13.211  12.109    40.0  300559
Processing: 300538
                  ope

             open  close   high    low    volume    code
date                                                    
2015-10-13  7.935  7.954  8.029  7.907  664182.0  600048
              open   close    high     low   volume    code
date                                                       
2014-12-12  10.329  10.476  10.511  10.277  48214.0  002278                   open  close   high    low  volume    code
date                                                        
2018-05-21 14:55  19.89  19.87  19.89  19.84  2865.0  603157

Processing: 603601              open   close    high     low    volume    code
date                                                        
2015-04-13  17.567  17.792  18.071  17.129  243332.0  300086

              open   close    high     low   volume    code
date                                                       
2015-04-22  19.871  20.118  20.477  19.043  44775.0  300081
                   open  close   high    low  volume    code
date                   


Processing: 601155
                  open  close   high    low  volume    code
date                                                       
2018-05-21 14:55  59.3  59.23  59.32  59.19  1210.0  600809
Processing: 300577                  open  close  high   low  volume    code
date                                                     
2018-05-21 14:55  4.09   4.09   4.1  4.09   723.0  000557

Processing: 603199
              open  close    high    low    volume    code
date                                                      
2015-10-14  15.175  14.66  15.561  14.42  311635.0  002363
            open  close   high    low    volume    code
date                                                   
2015-10-22  7.09  7.304  7.402  6.905  665032.0  002244
             open  close  high    low   volume    code
date                                                  
2015-10-15  12.93   13.5  13.5  12.85  33374.0  600962
                   open  close   high    low  volume    code
date             

            open  close  high  low  volume    code
date                                              
2017-12-18   6.7   8.04  8.04  6.7   403.0  603477                  open  close   high    low  volume    code
date                                                       
2018-05-21 14:55  11.2  11.18  11.25  11.14  5083.0  002683
Processing: 300712

              open   close    high     low    volume    code
date                                                        
2015-07-17  40.207  42.011  43.137  39.071  165419.0  300216
                  open  close  high   low   volume    code
date                                                      
2018-05-21 14:55  7.63   7.65  7.67  7.63  10732.0  000961
Processing: 000567
              open   close   high     low   volume    code
date                                                      
2015-10-22  19.645  20.542  20.71  19.422  24929.0  300445
              open   close    high     low   volume    code
date                            

...

            open  close  high   low    volume    code
date                                                 
2014-04-15  1.99   1.96  1.99  1.95  412088.0  000629
             open  close   high    low   volume    code
date                                                   
2015-09-28  8.356   8.67  8.738  8.069  26452.0  300390
                   open  close   high    low  volume    code
date                                                        
2018-05-21 14:55  16.96   17.0  17.01  16.96  2215.0  300178
                   open  close   high    low  volume    code
date                                                        
2018-05-21 14:55  17.53  17.53  17.54  17.51  3210.0  002202
             open  close   high    low   volume    code
date                                                   
2014-01-03  8.281  7.992  8.281  7.906  84813.0  002129
                   open  close   high    low  volume    code
date                                                        
2018-05-09 14:

  File "/home/samx/miniconda/lib/python3.6/site-packages/pandas/core/internals.py", line 3838, in get
    loc = self.items.get_loc(item)
  File "/home/samx/miniconda/lib/python3.6/site-packages/pandas/core/indexes/base.py", line 2524, in get_loc
    return self._engine.get_loc(self._maybe_cast_indexer(key))
  File "pandas/_libs/index.pyx", line 117, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/index.pyx", line 139, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/hashtable_class_helper.pxi", line 1265, in pandas._libs.hashtable.PyObjectHashTable.get_item
  File "pandas/_libs/hashtable_class_helper.pxi", line 1273, in pandas._libs.hashtable.PyObjectHashTable.get_item
KeyError: 'date'
Traceback (most recent call last):
  File "/home/samx/miniconda/lib/python3.6/site-packages/pandas/core/indexes/base.py", line 2522, in get_loc
    return self._engine.get_loc(key)
  File "pandas/_libs/index.pyx", line 117, in pandas._libs.index.IndexEngine.get_loc
  File "pa

             open   close    high     low   volume    code
date                                                      
2015-04-15  23.79  24.489  26.082  23.486  70295.0  002592             open  close   high    low   volume    code
date                                                   
2014-02-07  4.192   4.23  4.249  4.155  17265.0  600666


                   open  close   high    low  volume    code
date                                                        
2018-05-21 14:55  37.43  37.53  37.54  37.43  1515.0  601012
              open   close    high     low    volume    code
date                                                        
2015-10-15  15.201  15.416  15.416  14.898  184848.0  300274
            open  close  high   low   volume    code
date                                                
2013-09-12  6.34   6.45  6.46  6.33  98401.0  002070                   open  close   high    low  volume    code
date                                                        
2018-05-

  File "pandas/_libs/index.pyx", line 117, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/index.pyx", line 139, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/hashtable_class_helper.pxi", line 1265, in pandas._libs.hashtable.PyObjectHashTable.get_item
  File "pandas/_libs/hashtable_class_helper.pxi", line 1273, in pandas._libs.hashtable.PyObjectHashTable.get_item
KeyError: 'date'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/samx/miniconda/lib/python3.6/site-packages/threadpool.py", line 158, in run
    result = request.callable(*request.args, **request.kwds)
  File "<ipython-input-1-90f8b836df83>", line 31, in obtain_daily_data
    day_df = pre_process(ts.get_k_data(sk_code))
  File "<ipython-input-1-90f8b836df83>", line 21, in pre_process
    df.set_index('date', inplace=True)
  File "/home/samx/miniconda/lib/python3.6/site-packages/pandas/core/frame.py", line 3146, in set_index
 

EmptyDataError: No columns to parse from file

In [4]:
import datetime
now = datetime.datetime.now()
today_end2 = datetime.datetime(now.year, now.month, now.day, 15, 1)
tomorrow_end2 = today_end2 + datetime.timedelta(days=1)
today_end2, tomorrow_end2

(datetime.datetime(2017, 10, 16, 15, 1),
 datetime.datetime(2017, 10, 17, 15, 1))

In [2]:
  now = datetime.datetime.now()

In [6]:

today_end2 = datetime.datetime(now.year, now.month, now.day, 15, 1)
(now-today_end2).days

0

In [3]:
now

datetime.datetime(2017, 12, 11, 20, 30, 28, 861501)

In [5]:
get_daily_chart()

44:57
condition 1:000532
condition 1:000735
condition 1:002161
condition 1:002607
condition 1:002782
condition 1:300490
condition 1:300491
condition 1:300549
condition 1:300647
condition 1:300745
condition 1:600433
condition 1:600569
condition 1:601313
condition 1:601619
condition 1:603486
condition 2:002332
condition 2:002485
condition 2:002881
condition 2:002889
condition 2:300551
condition 2:300578
condition 2:603535
condition 2:603607
condition 2:603999
['000532', '000735', '002161', '002607', '002782', '300490', '300491', '300549', '300647', '300745', '600433', '600569', '601313', '601619', '603486', '002332', '002485', '002881', '002889', '300551', '300578', '603535', '603607', '603999']
condition 1:002458
condition 1:300490
condition 1:300647
condition 1:300648
condition 1:300682
condition 1:603045
condition 1:603648
condition 2:300710
condition 3:002290
condition 3:300578
condition 3:603535
['002458', '300490', '300647', '300648', '300682', '603045', '603648', '300710', '002290

In [ ]:
getDailyData()

folder already existed


In [ ]:
get_daily_chart()